# Refinitiv Data Platform Library for Python
## Delivery - EndPoint - Overview

This notebook provides an overview of using the Delivery Layer's Endpoint interface to retrieve Refinitiv Data Platform content directly from an Endpoint.

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
import refinitiv.data as rd
from refinitiv.data.delivery import endpoint_request
import datetime
import json
import asyncio
import pandas as pd

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
rd.open_session()

<refinitiv.data.session.platform.Definition object at 0x17fd0568a00 {session_name='rdp'}>

## Defining an Endpoint
Defining an Endpoint
At the heart of the Endpoint Request interfaces, is the data endpoint URL.  For example, the following data services are represented by their endpoint URLs:

* Historical Pricing: https://api.refinitiv.com/data/historical-pricing/v1/views/events
* News Headlines: https://api.refinitiv.com/data/news/v1/headlines 
* Symbology Lookup: https://api.refinitiv.com/discovery/symbology/v1/lookup 
* Basic ESG scores: https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic

When creating a request to retrieve data from an endpoint, the first step is to specify the endpoint URL, for example:

In [4]:
esg_url = 'https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic'

### Defining Endpoint Request Properties
Depending on the service and how you choose to prepare your endpoint URL, the interfaces support the ability to specify additional properties to be included within your request.  The following request features are available:

* Query Parameters Endpoint: URLs supporting query strings are represented as field=value pairs.
Eg: https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic?universe=IBM.N

* Path Parameters : In some cases, URLs define parameters that are embedded within the URL.  When defining the URL as part of your application, you must enclose the path parameter(s) within curly braces.
Eg: https://api.refinitiv.com/data/historical-pricing/v1/views/events/{universe}

* Header Parameters : Endpoints that enforce different content types or formats, typically require the specification within the header portion of the HTTP request.  For example, the endpoint to retrieve a news story offers the ability to specify the response format as either "text/html" or "application/json", which is specified within the HTTP header of the request.

* Body Parameters : Many of the RDP endpoints support the HTTP POST defining the ability to specify a body of data in the request.  The libraries support the ability to specify JSON details within the support endpoint request.  

The following simple code segments demonstrate the above mechanisms.

**Query Parameter:**

In [5]:
# Example request with Query Parameter - ESG
request_definition = rd.delivery.endpoint_request.Definition(
    url = esg_url,
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    query_parameters = {"universe": "IBM.N"}
)
response = request_definition.get_data()
response.data.raw

https://api.refinitiv.com https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic None {'universe': 'IBM.N'}


{'links': {'count': 1},
 'variability': 'fixed',
 'universe': [{'Instrument': 'IBM.N',
   'Company Common Name': 'International Business Machines Corp',
   'Organization PermID': '4295904307',
   'Reporting Currency': 'USD'}],
 'data': [['IBM.N',
   '2020-12-31',
   96.625022435838,
   51.5117088466531,
   78.9800773596806,
   98.9825581395348,
   99.3093922651933,
   53.8135593220339,
   99.6099843993759,
   83.3850931677018,
   99.4539781591263,
   98.1574539363484,
   39.6890096618357,
   78.06461352657,
   70.795847750865,
   73.3005404816844,
   100,
   '2021-08-28T00:00:00']],
 'messages': {'codes': [[-1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1]],
  'descriptions': [{'code': -1, 'description': 'ok'}]},
 'headers': [{'name': 'instrument',
   'title': 'Instrument',
   'type': 'string',
   'description': 'The requested Instrument as defined by the user.'},
  {'name': 'periodenddate',
   't

**Path Parameter:**

In [6]:
# Example request with Path parameter - News Story
# Specify our Endpoint URL
story_url = 'https://api.refinitiv.com/data/news/v1/stories/{storyId}'
# Specify our path parameter value
#storyId = "urn:newsml:reuters.com:20210330:nHKS9WjjsZ:2"
storyId = "urn:newsml:reuters.com:20210805:nZON003MH6:1"
request_definition = rd.delivery.endpoint_request.Definition(
    url = story_url,
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    path_parameters = {"storyId": storyId}
)
response = request_definition.get_data()
response.data.raw

https://api.refinitiv.com https://api.refinitiv.com/data/news/v1/stories/{storyId} {'storyId': 'urn:newsml:reuters.com:20210805:nZON003MH6:1'} None


{'newsItem': {'_conformance': 'power',
  '_guid': 'tag:reuters.com,2021:newsml_ZON003MH6',
  '_standard': 'NewsML-G2',
  '_standardversion': '2.18',
  '_version': 2,
  '_xml:lang': 'fr',
  'catalogRef': [{'_href': 'http://xml.media.reuters.com/g2-standards/catalogs/ReutersMedia_G2-Standards-Catalog_v1.xml'}],
  'rightsInfo': [{'_xml:lang': 'en',
    'copyrightHolder': {'_literal': 'Thomson Reuters'},
    'copyrightNotice': [{'$': '(c) Copyright Thomson Reuters 2021. Click For Restrictions - https://agency.reuters.com/en/copyright.html'}]}],
  'itemMeta': {'itemClass': {'_qcode': 'ninat:text', '_rtr:msgType': 'S'},
   'provider': {'_literal': 'reuters.com'},
   'versionCreated': {'$': '2021-08-05T12:48:13Z'},
   'firstCreated': {'$': '2021-08-05T12:48:13Z'},
   'pubStatus': {'_qcode': 'stat:usable'},
   'role': {'_qcode': 'itemRole:B'},
   'fileName': {'$': '2021-08-05T124813Z_2_ZON003MH6_RTRLYNXT_0_USA-CHOMAGE-INSCRIPTIONS.XML'},
   'profile': {'_versioninfo': '00.00.01', '$': 'SNI-Tex

**Header Parameter:**

In [7]:
# Example request with Path + Header parameter - News Story
# Specify our Endpoint URL
story_url = 'https://api.refinitiv.com/data/news/v1/stories/{storyId}'
# Specify our path parameter value
storyId = "urn:newsml:reuters.com:20210330:nHKS9WjjsZ:2"
request_definition = rd.delivery.endpoint_request.Definition(
    url = story_url,
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    path_parameters = {"storyId": storyId},
    header_parameters = {"accept": "text/html"}
)
response = request_definition.get_data()
response.data.raw

https://api.refinitiv.com https://api.refinitiv.com/data/news/v1/stories/{storyId} {'storyId': 'urn:newsml:reuters.com:20210330:nHKS9WjjsZ:2'} None


{'newsItem': {'_conformance': 'power',
  '_guid': 'tag:reuters.com,2021-03-30:newsml_HKS9WjjsZ',
  '_standard': 'NewsML-G2',
  '_standardversion': '2.18',
  '_version': 2,
  'catalogRef': [{'_href': 'http://xml.media.reuters.com/g2-standards/catalogs/ReutersMedia_G2-Standards-Catalog_v1.xml'}],
  'rightsInfo': [{'copyrightHolder': {'_literal': ''},
    'copyrightNotice': [{}]}],
  'itemMeta': {'itemClass': {'_qcode': 'ninat:text', '_rtr:msgType': 'R'},
   'provider': {'_literal': 'reuters.com'},
   'versionCreated': {'$': '2021-03-30T14:09:48.681Z'},
   'firstCreated': {'$': '2021-03-30T14:09:09.503Z'},
   'pubStatus': {'_qcode': 'stat:usable'},
   'role': {'_qcode': 'itemRole:N'},
   'title': [{'$': 'Shenwan Hongyuan Group Co Ltd - An announcement has just been published by the issuer in the Chinese section of this website, a corresponding version of which may or may not be published in this section(with URL)'}],
   'signal': [{'_qcode': 'edStat:U'}],
   'expires': [{'$': '2022-04-30T

**Body Parameter:**

In [8]:
# Example request with Body Parameter - Symbology Lookup
lookup_url = 'https://api.refinitiv.com/discovery/symbology/v1/lookup'
request_definition = rd.delivery.endpoint_request.Definition(
    url = lookup_url,
    method = rd.delivery.endpoint_request.RequestMethod.POST,
    body_parameters = {
        "from": [
            {
                "identifierTypes": [
                    "RIC"
                ],
                "values": [
                    "MSFT.O",
                    "IBM.N"
                ]
            }
        ],
        "to": [
            {
                "identifierTypes": [
                    "LEI",
                    "ISIN",
                    "ExchangeTicker"
                ]
            }
        ],
        "type": "auto"
    }
)
response = request_definition.get_data()
response.data.raw

https://api.refinitiv.com https://api.refinitiv.com/discovery/symbology/v1/lookup None None


{'data': [{'input': [{'value': 'IBM.N', 'identifierType': 'RIC'}],
   'output': [{'value': 'US4592001014', 'identifierType': 'ISIN'},
    {'value': 'IBM', 'identifierType': 'ExchangeTicker'},
    {'value': 'VGRQXHF3J8VDLUA7XE92', 'identifierType': 'LEI'}]},
  {'input': [{'value': 'MSFT.O', 'identifierType': 'RIC'}],
   'output': [{'value': 'INR2EJN1ERAN0W5ZP974', 'identifierType': 'LEI'},
    {'value': 'US5949181045', 'identifierType': 'ISIN'},
    {'value': 'MSFT', 'identifierType': 'ExchangeTicker'}]}],
 'requestId': '54c1f6ca-a991-4d59-8439-bbc75577d6aa',
 'effectiveAt': '2021-12-17T11:26:20.372Z',
 'messages': []}

### How to send several request with asyncio.gather

The above examples represent simple requests to retrieve the data synchronously from the platform.  While this workflow may be suitable for simple applications or when demonstrating functionality, multi-threaded applications, including those that are GUI/Windows-based, will require asynchronous behavior. For example, asyncio.gather allows us to send several request simultaneously (multi-threaded requests).

#### Example: request ESG data for two instruments

In [9]:
request_definition_1 = rd.delivery.endpoint_request.Definition(
    url = esg_url,
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    query_parameters = {"universe": "IBM.N"}
)
request_definition_2 = rd.delivery.endpoint_request.Definition(
    url = esg_url,
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    query_parameters = {"universe": "BT.L"}
)

response = request_definition_1.get_data_async()
tasks = asyncio.gather(
    request_definition_1.get_data_async(),
    request_definition_2.get_data_async()
)

# Wait for all tasks to be completed
asyncio.get_event_loop().run_until_complete(tasks)
ibm,bt = tasks._result

https://api.refinitiv.com https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic None {'universe': 'IBM.N'}
https://api.refinitiv.com https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic None {'universe': 'BT.L'}


### Checking the Response Data
When using ***async*** calls, we should check the response's ***is_success*** flag to ensure it worked. If it failed, the ***http_status*** should provide a reason for the failure.

In [12]:
def display_reponse(response):
    print(response)
    print("\nReponse received for", response.closure)
    if response.is_success:
        display(response.data.raw)
    else:
        print(response.http_status)

display_reponse(ibm)
display_reponse(bt)


Reponse received for None


{'links': {'count': 1},
 'variability': 'fixed',
 'universe': [{'Instrument': 'IBM.N',
   'Company Common Name': 'International Business Machines Corp',
   'Organization PermID': '4295904307',
   'Reporting Currency': 'USD'}],
 'data': [['IBM.N',
   '2020-12-31',
   96.625022435838,
   51.5117088466531,
   78.9800773596806,
   98.9825581395348,
   99.3093922651933,
   53.8135593220339,
   99.6099843993759,
   83.3850931677018,
   99.4539781591263,
   98.1574539363484,
   39.6890096618357,
   78.06461352657,
   70.795847750865,
   73.3005404816844,
   100,
   '2021-08-28T00:00:00']],
 'messages': {'codes': [[-1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1]],
  'descriptions': [{'code': -1, 'description': 'ok'}]},
 'headers': [{'name': 'instrument',
   'title': 'Instrument',
   'type': 'string',
   'description': 'The requested Instrument as defined by the user.'},
  {'name': 'periodenddate',
   't


Reponse received for None


{'links': {'count': 1},
 'variability': 'fixed',
 'universe': [{'Instrument': 'BT.L',
   'Company Common Name': 'BT Group PLC',
   'Organization PermID': '4295895409',
   'Reporting Currency': 'GBP'}],
 'data': [['BT.L',
   '2021-03-31',
   69.9430781675061,
   82.8899893433805,
   85.9961093278864,
   94.3333333333333,
   98.3221476510067,
   62.3809523809523,
   80.7262569832402,
   50.735294117647,
   80.7262569832402,
   71.6763005780346,
   80.2631578947368,
   86.7647058823529,
   90.2120717781402,
   76.6378680228668,
   81.063,
   'Lloyds Register Quality Assurance Ltd LRQA',
   '2021-12-09T00:00:00']],
 'messages': {'codes': [[-1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1,
    -1]],
  'descriptions': [{'code': -1, 'description': 'ok'}]},
 'headers': [{'name': 'instrument',
   'title': 'Instrument',
   'type': 'string',
   'description': 'The requested Instrument as defined by the user.

In [13]:
rd.close_session()